In [2]:
pip install deap


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
     ----- ---------------------------------- 0.1/1.1 MB 2.8 MB/s eta 0:00:01
     ------------ --------------------------- 0.3/1.1 MB 4.1 MB/s eta 0:00:01
     ------------- -------------------------- 0.4/1.1 MB 3.7 MB/s eta 0:00:01
     ------------------------------- -------- 0.8/1.1 MB 4.8 MB/s eta 0:00:01
     ------------------------------------- -- 1.0/1.1 MB 4.9 MB/s eta 0:00:01
     ---------------------------------------- 1.1/1.1 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for deap: filename=deap-1.4.1-py3-none-any.whl size=97349 sha256=138f99c6a7e2b8f6d13f1aef79a821d0bfb2b3cc1936daffcfdea5f7131f83ae
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\f8\64\b8\65eacfbff3024ae2e2beb22e691d5c8abb89fbd863b8049b5f
Succes

In [6]:
pip install pandas


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
import random
from deap import base, creator, tools, algorithms
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import precision_recall_curve, auc
import pandas as pd

# Load the dataset from a CSV file
df = pd.read_csv('creditcard.csv')

# Split the dataset into features (X) and the target variable (y)
X = df.drop('Class', axis=1).values  # Assuming 'Class' is the target variable
y = df['Class'].values

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the evaluation function (fitness function) using XGBoost
def evaluate(individual, X, y):
    # Create a mask to select features
    mask = [bool(bit) for bit in individual]
    X_selected = X[:, mask]
    
     # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)
    
    # Train an XGBoost classifier
    clf = xgb.XGBClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_preds = clf.predict(X_test)
    
    # Calculate precision-recall AUC
    y_score = clf.predict_proba(X_test)[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, y_score)
    auc_score = auc(recall, precision)
    
    return auc_score,

# Create a fitness function (maximize AUC)
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Genetic algorithm toolbox setup
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=30)  # 30 features
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate, X=X_train, y=y_train)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# Create the population
population = toolbox.population(n=10)

# Define the number of generations
n_generations = 10

# Run the genetic algorithm
for gen in range(n_generations):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=0.2)
    fits = toolbox.map(toolbox.evaluate, offspring)
    
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    
    population = toolbox.select(offspring, k=len(population))

# Retrieve the best individual (features)
best_individual = tools.selBest(population, k=1)[0]
best_feature_mask = [bool(bit) for bit in best_individual]

# Use the best feature subset for model training and testing
X_selected = X_train[:, best_feature_mask]
X_test_selected = X_test[:, best_feature_mask]

clf = xgb.XGBClassifier(n_estimators=100, random_state=42)
clf.fit(X_selected, y_train)
y_preds = clf.predict(X_test_selected)

# Evaluate the final model with the best feature subset
y_score = clf.predict_proba(X_test_selected)[:, 1]
precision, recall, _ = precision_recall_curve(y_test, y_score)
auc_score = auc(recall, precision)

# Print the AUC score of the final model
print(f"Final AUC Score: {auc_score}")

# The best_feature_mask contains the selected features for the final model



C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "

KeyboardInterrupt



In [9]:
import random
from deap import base, creator, tools, algorithms
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import precision_recall_curve, auc
import pandas as pd

# Load the dataset from a CSV file
df = pd.read_csv('creditcard.csv')

# Split the dataset into features (X) and the target variable (y)
X = df.drop('Class', axis=1).values  # Assuming 'Class' is the target variable
y = df['Class'].values

In [10]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [24]:
# Define the evaluation function (fitness function) using XGBoost
def evaluate(individual, X, y):
    # Create a mask to select features
    mask = [bool(bit) for bit in individual]
    X_selected = X[:, mask]
    
     # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)
    
    # Train an XGBoost classifier
    clf = xgb.XGBClassifier(n_estimators=10, random_state=42)
    clf.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_preds = clf.predict(X_test)
    
    # Calculate precision-recall AUC
    y_score = clf.predict_proba(X_test)[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, y_score)
    auc_score = auc(recall, precision)
    
     # Calculate accuracy
    accuracy = accuracy_score(y_test, y_preds)
    
    # Calculate F1 score
    f1 = f1_score(y_test, y_preds)
    
    return auc_score,accuracy, f1

In [12]:
# Create a fitness function (maximize AUC)
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [19]:
# Genetic algorithm toolbox setup
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=30)  # 30 features
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate, X=X_train, y=y_train)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)


In [20]:
# Create the population
population = toolbox.population(n=10)

# Define the number of generations
n_generations = 5


In [15]:
# Run the genetic algorithm
for gen in range(n_generations):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=0.2)
    fits = toolbox.map(toolbox.evaluate, offspring)
    
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    
    population = toolbox.select(offspring, k=len(population))


KeyboardInterrupt



In [ ]:

# Retrieve the best individual (features)
best_individual = tools.selBest(population, k=1)[0]
best_feature_mask = [bool(bit) for bit in best_individual]

# Use the best feature subset for model training and testing
X_selected = X_train[:, best_feature_mask]
X_test_selected = X_test[:, best_feature_mask]

clf = xgb.XGBClassifier(n_estimators=100, random_state=42)
clf.fit(X_selected, y_train)
y_preds = clf.predict(X_test_selected)

In [ ]:
# Evaluate the final model with the best feature subset
y_score = clf.predict_proba(X_test_selected)[:, 1]
precision, recall, _ = precision_recall_curve(y_test, y_score)
auc_score = auc(recall, precision)

# Print the AUC score of the final model
print(f"Final AUC Score: {auc_score}")

# The best_feature_mask contains the selected features for the final model

In [21]:
# Run the genetic algorithm
for gen in range(n_generations):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=0.2)
    fits = toolbox.map(toolbox.evaluate, offspring)
    
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    
    population = toolbox.select(offspring, k=len(population))

# Retrieve the best individual (features)
best_individual = tools.selBest(population, k=1)[0]
best_feature_mask = [bool(bit) for bit in best_individual]

# Use the best feature subset for model training and testing
X_selected = X_train[:, best_feature_mask]
X_test_selected = X_test[:, best_feature_mask]

clf = xgb.XGBClassifier(n_estimators=100, random_state=42)
clf.fit(X_selected, y_train)
y_preds = clf.predict(X_test_selected)

# Evaluate the final model with the best feature subset
y_score = clf.predict_proba(X_test_selected)[:, 1]
precision, recall, _ = precision_recall_curve(y_test, y_score)
auc_score = auc(recall, precision)

# Print the AUC score of the final model
print(f"Final AUC Score: {auc_score}")

# The best_feature_mask contains the selected features for the final model


Final AUC Score: 0.8628450343680472


In [25]:
# Evaluate the final model with the best feature subset
accuracy = accuracy_score(y_test, y_preds)
f1 = f1_score(y_test, y_preds)

NameError: name 'accuracy_score' is not defined

# # In the provided code example, we are using the XGBoost classifier as the machine learning model to evaluate the feature subsets selected by the genetic algorithm. The genetic algorithm is used to perform feature selection, not to build a classifier. The genetic algorithm is used to select the most relevant features for the classification problem.

# we are setting up the genetic algorithm using the DEAP (Distributed Evolutionary Algorithms in Python) library. DEAP provides a framework for creating and customizing genetic algorithms.

In [28]:
"""import random
from deap import base, creator, tools, algorithms
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import precision_recall_curve, auc
import pandas as pd

# Load the dataset from a CSV file
df = pd.read_csv('your_dataset.csv')

# Split the dataset into features (X) and the target variable (y)
X = df.drop('Class', axis=1).values  # Assuming 'Class' is the target variable
y = df['Class'].values

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

"""

from sklearn.metrics import accuracy_score, f1_score 

# Define the evaluation function (fitness function) using XGBoost
def evaluate(individual, X, y):
    # Create a mask to select features
    mask = [bool(bit) for bit in individual]
    X_selected = X[:, mask]

    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

    # Train an XGBoost classifier
    clf = xgb.XGBClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    # Make predictions on the test data
    y_preds = clf.predict(X_test)

    # Calculate precision-recall AUC
    y_score = clf.predict_proba(X_test)[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, y_score)
    auc_score = auc(recall, precision)

    return auc_score,

# Create a fitness function (maximize AUC)
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Genetic algorithm toolbox setup
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=30)  # 30 features
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate, X=X_train, y=y_train)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# Create the population
population = toolbox.population(n=10)

# Define the number of generations
n_generations = 10

# Run the genetic algorithm
for gen in range(n_generations):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=0.2)
    fits = toolbox.map(toolbox.evaluate, offspring)

    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit

    population = toolbox.select(offspring, k=len(population))

# Retrieve the best individual (features)
best_individual = tools.selBest(population, k=1)[0]

# The best_feature_mask contains the selected features for the final model
best_feature_mask = [bool(bit) for bit in best_individual]

# Use the best feature subset for model training and testing
X_selected = X_train[:, best_feature_mask]
X_test_selected = X_test[:, best_feature_mask]

clf = xgb.XGBClassifier(n_estimators=100, random_state=42)
clf.fit(X_selected, y_train)
y_preds = clf.predict(X_test_selected)

# Evaluate the final model with the best feature subset
y_score = clf.predict_proba(X_test_selected)[:, 1]
precision, recall, _ = precision_recall_curve(y_test, y_score)
auc_score = auc(recall, precision)

# Calculate accuracy and F1 score
accuracy = accuracy_score(y_test, y_preds)
f1 = f1_score(y_test, y_preds)

# Print the AUC score, accuracy, and F1 score of the final model
print(f"Final AUC Score: {auc_score}")
print(f"Final Accuracy Score: {accuracy}")
print(f"Final F1 Score: {f1}")


C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Final AUC Score: 0.8180393825294777
Final Accuracy Score: 0.9994908886626171
Final F1 Score: 0.8379888268156425
